In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False


In [101]:
df = pd.read_csv('../../data/team/Pollution 전처리/해양환경공단_해양폐기물정화사업 현황_20181231.csv', encoding = 'ANSI', engine = 'python')
df.to_csv('../../data/team/폐기물정화사업.csv',encoding = 'utf-8',index=False)

In [102]:
df_csv('../../data/team/폐기물정화사업.csv',encoding = 'utf-8',index=False)
df = df.rename(columns={'사업명(위치)':'위치','수거물량 실적(톤)':'수거','정화면적 실적(ha)':'정화면적'})
df

,년도,위치,수거,정화면적,사업기간
0,2018,보령시 보령항,65.59,3212.0,6.25~9.22
1,2018,사천시 삼천포항,69.29,23.0,3.16~5.14
2,2018,포항시 포항항,67.75,2325.0,6.19~8.17
3,2018,창원시 마산항,51.72,400.0,6.29~8.27
4,2018,창원시 진해항,61.60,500.0,3.16~7.15
...,...,...,...,...,...
82,2015,경북 울릉항,29.07,33.0,10.26~12.09
83,2015,제주 서귀포항,222.12,15.8,10.26~12.09
84,2015,제주 제주항,74.12,9.2,10.26~12.09
85,2015,신안 우이도항,125.52,190.0,11.16~12.31


In [ ]:
df['정화면적'] = df['정화면적'].astype(float)



In [ ]:
df['시군'] = '' 

df['시군'] = df['위치'].str.split(' ').str[0]
df

In [124]:
df

,년도,위치,수거,정화면적,사업기간,생태구역,시군
0,2018,보령시 보령항,65.59,3212.0,6.25~9.22,대한해협,보령시
1,2018,사천시 삼천포항,69.29,23.0,3.16~5.14,대한해협,사천시
2,2018,포항시 포항항,67.75,2325.0,6.19~8.17,동해,포항시
3,2018,창원시 마산항,51.72,400.0,6.29~8.27,대한해협,창원시
4,2018,창원시 진해항,61.60,500.0,3.16~7.15,대한해협,창원시
...,...,...,...,...,...,...,...
82,2015,경북 울릉항,29.07,33.0,10.26~12.09,동해,경북
83,2015,제주 서귀포항,222.12,15.8,10.26~12.09,대한해협,제주
84,2015,제주 제주항,74.12,9.2,10.26~12.09,대한해협,제주
85,2015,신안 우이도항,125.52,190.0,11.16~12.31,대한해협,신안


In [105]:
df_all = pd.read_csv('../../data/team/수질+수거량.csv')
df_all['구군']

0        거진
1        거진
2        속초
3        속초
4        속초
       ... 
7341    낙동강
7342    낙동강
7343    낙동강
7344    낙동강
7345    낙동강
Name: 구군, Length: 7346, dtype: object

In [106]:
li_a = list(df_all['구군'])
li_b = []
for i in li_a:
    a = i.replace(' ','')
    li_b.append(a)
df_all['구군'] = li_b

In [129]:
gu = df_all['구군'].unique()

In [134]:
# 기존파일의 생태구역을 먼저 넣은 뒤 부족한 지역을 지정해서 넣음
# 기준은 해양환경관리법의 기준에 따름
for i in range(len(df)):
    for j in gu:
        if df.loc[i,'위치'].find(j) != -1:
            df.loc[i,'생태구역'] = df_all[df_all['구군'] == j]['생태구역'].iloc[0]

for i in df[df['생태구역'].isnull()].index:
    al = df.loc[i,'위치']
    if (al.find('삼천포') != -1)|(al.find('거제') != -1)|(al.find('하동') != -1)|(al.find('경남') != -1):
        df.loc[i,'생태구역'] = '대한해협'
        
    elif (al.find('서천') != -1)|(al.find('당진') != -1)|(al.find('충남') != -1)|(al.find('영종도') != -1)|(al.find('평택') != -1):
        df.loc[i,'생태구역'] = '서해중부'
        
    elif (al.find('당진') != -1)|(al.find('장흥') != -1)|(al.find('전북') != -1)|(al.find('영암') != -1)|(al.find('전남') != -1)|(al.find('흑산도') != -1):
        df.loc[i,'생태구역'] = '서남해역'
        
    elif (al.find('경북') != -1)|(al.find('독도') != -1)|(al.find('포항') != -1):
        df.loc[i,'생태구역'] = '동해'

In [200]:
df1 = df.groupby('시군').mean()

In [219]:
df2 = df.groupby(['년도','생태구역']).mean()

df1_2 = pd.DataFrame(columns=['년','생태구역','수거량','면적'])
idx = 0
for i in df2.index:
    df1_2.loc[idx,'년'] = i[0]
    df1_2.loc[idx,'생태구역'] = i[1]
    df1_2.loc[idx,'수거량'] = df2.loc[i[0],i[1]][0]
    df1_2.loc[idx,'면적'] = df2.loc[i[0],i[1]][1]
    idx +=1
    
    

In [220]:
avg = df1_2[df1_2['생태구역'] == '제주'].mean()
df1_2['년'] = df1_2['년'].astype(float)

In [254]:
df1_2.loc[18] = [2016,'제주',avg[1], avg[2]]
df1_2.loc[19] = [2017,'제주',avg[1], avg[2]]

In [257]:
df_all['수거량2']=''
df_all['정화면적']=''

for i in df_all.index:
    
    year = round(df_all.loc[i,'년월']/100)
    area = df_all.loc[i,'생태구역'] 
    
    for j in df1_2['년'].unique():
        for z in df1_2['생태구역'].unique():
            if (year==j)&(area==z):
                tem = df1_2[(df1_2['년'] ==j)&(df1_2['생태구역']==z)]
                df_all.loc[i,'수거량2'] = tem['수거량'].iloc[0]
                df_all.loc[i,'정화면적'] = tem['면적'].iloc[0]
   
            

In [ ]:
for i in df_all.index:
    year = int(df_all.loc[i,'년월']/100)
    area = df_all.loc[i,'구군'] 
    for j in df.index:
        if (year == df.loc[j,'년도'])&(df.loc[j,'위치'].find(area) != -1):
            df_all.loc[i,'수거량2'] = df.loc[j,'수거']
            df_all.loc[i,'정화면적'] = df.loc[j,'정화면적']
  

In [273]:
mini = df_all['수거량2'].min()
maxi = df_all['수거량2'].max()

df_all['수거량2'] = (df_all['수거량2']-mini)/(maxi-mini)

mini = df_all['정화면적'].min()
maxi = df_all['정화면적'].max()

df_all['정화면적'] = (df_all['정화면적']-mini)/(maxi-mini)

In [275]:
df_all.to_csv('../../data/team/수질+수거량+정화.csv', encoding='utf-8', index=False)
df_all.to_csv('../../data/team/수질+수거량+정화_R.csv', encoding='ANSI', index=False)

In [276]:
df_all

,년월,생태구역,연안명칭,St.\nNo\n,해수수질기준,표층수온,저층수온,표층염분,저층염분,표층pH,...,표층SS,저층SS,표층Chl-a,저층Chl-a,표층투명도,구군,Ds,수거량,수거량2,정화면적
0,201702.0,동해,거진연안,1,1,0.356804,0.346780,0.990086,0.349752,0.143968,...,0.018180,0.009271,0.005299,0.005346,0.494297,거진,NaN,0.053470,0.142278,0.00687333
1,201702.0,동해,거진연안,2,1,0.340376,0.336072,0.990305,0.349964,0.144314,...,0.023290,0.019876,0.000000,0.005476,0.342205,거진,NaN,0.053470,0.142278,0.00687333
2,201702.0,동해,속초연안,1,1,0.345105,0.343661,0.988656,0.349111,0.140853,...,0.020735,0.011496,0.004899,0.005065,0.418251,속초,NaN,0.053470,0.142278,0.00687333
3,201702.0,동해,속초연안,2,1,0.348785,0.349219,0.989568,0.349601,0.142584,...,0.019064,0.014166,0.003281,0.005736,0.494297,속초,NaN,0.053470,0.142278,0.00687333
4,201702.0,동해,속초연안,3,1,0.346118,0.344593,0.989458,0.349505,0.143622,...,0.023585,0.006378,0.000000,0.004286,0.380228,속초,NaN,0.053470,0.142278,0.00687333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7341,201611.0,대한해협,낙동강하구,6,4,0.521305,0.555194,0.721092,0.324401,0.141199,...,0.035770,0.036193,0.007302,0.007835,0.049430,낙동강,7.945174,0.554812,0.264991,0.0213439
7342,201611.0,대한해협,낙동강하구,7,3,0.524070,0.536896,0.780402,0.317050,0.143622,...,0.019261,0.024326,0.006070,0.003160,0.053232,낙동강,8.070723,0.554812,0.264991,0.0213439
7343,201611.0,대한해협,낙동강하구,8,3,0.513596,0.543726,0.692463,0.323414,0.143968,...,0.018868,0.019283,0.006840,0.004481,0.049430,낙동강,8.007933,0.554812,0.264991,0.0213439
7344,201611.0,대한해협,낙동강하구,9,2,0.556069,0.554851,0.930226,0.326625,0.142930,...,0.017099,0.015575,0.003620,0.004156,0.057034,낙동강,7.937472,0.554812,0.264991,0.0213439
